In [ ]:
import time
start_time = time.time()

In [ ]:
!git clone -b final_ts_mixer https://github.com/jeevitharodrigues/ts_mixer.git
!mv /content/ts_mixer/* /content
!rm -rf /content/ts_mixer

Cloning into 'tsmixer'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 111 (delta 56), reused 92 (delta 37), pack-reused 0
Receiving objects: 100% (111/111), 2.61 MiB | 4.87 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [ ]:
import argparse
import os
import time
import pandas as pd
import numpy as np
import tensorflow as tf
import glob
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from data_loader import TSFDataLoader  # You may need to adapt this import
import models  # You may need to adapt this import
from models import tsmixer
from sklearn.metrics import mean_squared_error, mean_absolute_error
root_mean_squared_error = lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred))
import random
import warnings
warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')

In [ ]:
args_delete_checkpoint = False
args_checkpoint_dir = './checkpoints/'
args_result_path = 'result.csv'
args_seed = 1

In [ ]:
tf.random.set_seed(args_seed)
np.random.seed(args_seed)
random.seed(args_seed)
tf.keras.utils.set_random_seed(args_seed)

In [ ]:
train_mse = None
train_rmse = None
train_median = None
train_mae = None
val_mse = None
val_rmse = None
val_median = None
val_mae = None
test_mse = None
test_rmse = None
test_median = None
test_mae = None
full_mse = None
full_rmse = None
full_median = None
full_mae = None
train_data = None
val_data = None
test_data = None
full_data = None
elasped_training_time = None
n_feature = None
target_slice = None

In [ ]:
def create_folder(folder_path):
    try:
        os.makedirs(folder_path)
    except FileExistsError:
        pass

In [ ]:
def get_date():
  test_data = pd.read_csv(f"dataset/{args_data}.csv")
  n = test_data.shape[0]

  train_len = round(args_train_len - 0.05,2)
  test_len = round(1 - args_train_len,2)

  train_end = int(n * train_len)
  val_end = n - int(n * test_len)
  test_end = n

  df_full = test_data[1:].set_index('date')
  train_df = test_data[1:train_end].set_index('date')
  val_df = test_data[train_end - args_seq_len + 1 : val_end].set_index('date')
  test_df = test_data[val_end - args_seq_len + 1 : test_end].set_index('date')

  print(f"df_full len - {df_full.shape[0]}")
  print(f"train_df len - {train_df.shape[0]}")
  print(f"val_df len - {val_df.shape[0]}")
  print(f"test_df len - {test_df.shape[0]}")

  df_full_dates = df_full.index
  train_df_dates = train_df.index
  val_df_dates = val_df.index
  test_df_dates = test_df.index

  return (df_full_dates, train_df_dates, val_df_dates, test_df_dates)

In [ ]:
def get_input_and_target_df(pred_data):
  # Assuming pred_data is your dataset
  input_data_list = []
  target_data_list = []

  for input_tensor, target_tensor in pred_data:
      input_data = input_tensor.numpy()
      target_data = target_tensor.numpy()

      input_data_list.append(input_data)
      target_data_list.append(target_data)

  # Combine all input_data into one NumPy array
  combined_input_data = np.concatenate(input_data_list, axis=0)

  # Combine all target_data into one NumPy array
  combined_target_data = np.concatenate(target_data_list, axis=0)

  # Convert NumPy arrays to Pandas DataFrames
  input_df = pd.DataFrame(combined_input_data.reshape(-1, combined_input_data.shape[-1]), columns=[f'Input_{i}' for i in range(combined_input_data.shape[-1])])
  target_df = pd.DataFrame(combined_target_data.reshape(-1, combined_target_data.shape[-1]), columns=[f'Target_{i}' for i in range(combined_target_data.shape[-1])])

  # Print the shapes of the resulting dataframes
  print("Input Data Shape:", input_df.shape)
  print("Target Data Shape:", target_df.shape)

  return input_df,target_df

In [ ]:
def run_train_pred():
  global train_mse, train_rmse, train_mae

  pred_data = train_data

  build_model = tsmixer.build_model
  model = build_model(
      input_shape = (args_seq_len, n_feature),
      pred_len = args_pred_len,
      norm_type = args_norm_type,
      activation = args_activation,
      dropout = args_dropout,
      n_block = args_n_block,
      ff_dim = args_ff_dim,
      target_slice = target_slice,
  )

  checkpoint_path = os.path.join(args_checkpoint_dir, f'{exp_id}_best')

  model.load_weights(checkpoint_path)
  predictions_per_day = model.predict(pred_data)
  pred_g_val = [pday[0][0] for pday in predictions_per_day]

  print(f"Prediction Length for train data {len(pred_g_val)}")

  input_df, target_df = get_input_and_target_df(pred_data)
  # Given values
  substrings_to_check = ['g_cases_worldwide_2yr', 'g_cases_worldwide_3mth']

  if(any(substring in args_data for substring in substrings_to_check)):
    Y_true = list(input_df['Input_0'].values)
  else:
    Y_true = list(input_df['Input_500'].values)

  # calculated values
  Y_pred = pred_g_val

  Y_true.pop(0)
  Y_pred.pop(0)

  # Calculation of Mean Squared Error (MSE)

  train_mse = mean_squared_error(Y_true,Y_pred)
  train_rmse = root_mean_squared_error(Y_true,Y_pred)
  train_mae = mean_absolute_error(Y_true,Y_pred)

In [ ]:
def run_val_pred():
  global val_mse, val_rmse, val_mae

  pred_data = val_data

  build_model = tsmixer.build_model
  model = build_model(
      input_shape = (args_seq_len, n_feature),
      pred_len = args_pred_len,
      norm_type = args_norm_type,
      activation = args_activation,
      dropout = args_dropout,
      n_block = args_n_block,
      ff_dim = args_ff_dim,
      target_slice = target_slice,
  )

  checkpoint_path = os.path.join(args_checkpoint_dir, f'{exp_id}_best')

  model.load_weights(checkpoint_path)
  predictions_per_day = model.predict(pred_data)
  pred_g_val = [pday[0][0] for pday in predictions_per_day]

  print(f"Prediction Length for val data {len(pred_g_val)}")

  input_df, target_df = get_input_and_target_df(pred_data)
  # Given values
  substrings_to_check = ['g_cases_worldwide_2yr', 'g_cases_worldwide_3mth']

  if(any(substring in args_data for substring in substrings_to_check)):
    Y_true = list(input_df['Input_0'].values)
  else:
    Y_true = list(input_df['Input_500'].values)

  # calculated values
  Y_pred = pred_g_val

  Y_true.pop(0)
  Y_pred.pop(0)

  # Calculation of Mean Squared Error (MSE)

  val_mse = mean_squared_error(Y_true,Y_pred)
  val_rmse = root_mean_squared_error(Y_true,Y_pred)
  val_mae = mean_absolute_error(Y_true,Y_pred)

In [ ]:
def run_test_pred():
  global test_mse, test_rmse, test_mae
  pred_data = test_data

  build_model = tsmixer.build_model
  model = build_model(
      input_shape = (args_seq_len, n_feature),
      pred_len = args_pred_len,
      norm_type = args_norm_type,
      activation = args_activation,
      dropout = args_dropout,
      n_block = args_n_block,
      ff_dim = args_ff_dim,
      target_slice = target_slice,
  )

  checkpoint_path = os.path.join(args_checkpoint_dir, f'{exp_id}_best')

  model.load_weights(checkpoint_path)
  predictions_per_day = model.predict(pred_data)
  pred_g_val = [pday[0][0] for pday in predictions_per_day]

  print(f"Prediction Length for test data {len(pred_g_val)}")

  input_df, target_df = get_input_and_target_df(pred_data)
  # Given values
  substrings_to_check = ['g_cases_worldwide_2yr', 'g_cases_worldwide_3mth']

  if(any(substring in args_data for substring in substrings_to_check)):
    Y_true = list(input_df['Input_0'].values)
  else:
    Y_true = list(input_df['Input_500'].values)

  # calculated values
  Y_pred = pred_g_val

  Y_true.pop(0)
  Y_pred.pop(0)

  # Calculation of Mean Squared Error (MSE)

  test_mse = mean_squared_error(Y_true,Y_pred)
  test_rmse = root_mean_squared_error(Y_true,Y_pred)
  test_mae = mean_absolute_error(Y_true,Y_pred)

  dates = list(get_date()[3])
  dates.pop(0)

  data = {'date': pd.to_datetime(dates, format='%Y-%m-%d'), 'Y_true': Y_true, 'Y_pred': Y_pred}
  df = pd.DataFrame(data)
  interval = 30 if df.shape[0]>100 else 3
  percentile_x = np.percentile(df['date'], args_train_len*100)

  # Plot the data
  plt.figure(figsize=(10, 6))
  plt.plot(df['date'], df['Y_true'], label='Y_true', marker='o')
  plt.plot(df['date'], df['Y_pred'], label='Y_pred', marker='x')
  plt.axvline(percentile_x, color='red', linestyle='--', label=f'Vertical Line at {int(args_train_len*100)}%')
  plt.xlabel('Date')
  plt.ylabel('Values')
  plt.title(f'Y_true vs Y_pred for {args_data} {scalar_type_name}')
  plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
  plt.legend()
  plt.grid()
  plt.plot()
  plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=interval))  #change it to 30 if data size is more and 3 if data size is less
  plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

  # Specify the folder path and filename for saving the plot
  folder_path = f'Graphs/{train_test_size}/{scalar_type_name}/'
  file_name = f'{row+1}-{args_data}_test.png'

  create_folder(folder_path)

  # Combine folder path and file name
  file_path = folder_path + file_name

  # Save the plot to the specified file path
  plt.savefig(file_path)

In [ ]:
def run_full_pred():
  global full_mse, full_rmse, full_mae
  # pred_data = full_data

  # load datasets
  data_loader = TSFDataLoader(
      args_data,
      args_batch_size,
      args_seq_len,
      args_pred_len,
      args_feature_type,
      args_train_len,
      args_scaler_type,
      args_target,
  )

  pred_data = data_loader.get_full()

  build_model = tsmixer.build_model
  model = build_model(
      input_shape = (args_seq_len, n_feature),
      pred_len = args_pred_len,
      norm_type = args_norm_type,
      activation = args_activation,
      dropout = args_dropout,
      n_block = args_n_block,
      ff_dim = args_ff_dim,
      target_slice = target_slice,
  )

  checkpoint_path = os.path.join(args_checkpoint_dir, f'{exp_id}_best')

  model.load_weights(checkpoint_path)
  predictions_per_day = model.predict(pred_data)
  pred_g_val = [pday[0][0] for pday in predictions_per_day]

  print(f"Prediction Length for full data {len(pred_g_val)}")

  input_df, target_df = get_input_and_target_df(pred_data)
  # Given values
  substrings_to_check = ['g_cases_worldwide_2yr', 'g_cases_worldwide_3mth']

  if(any(substring in args_data for substring in substrings_to_check)):
    Y_true = list(input_df['Input_0'].values)
  else:
    Y_true = list(input_df['Input_500'].values)

  # calculated values
  Y_pred = pred_g_val

  Y_true.pop(0)
  Y_pred.pop(0)

  # Calculation of Mean Squared Error (MSE)

  full_mse = mean_squared_error(Y_true,Y_pred)
  full_rmse = root_mean_squared_error(Y_true,Y_pred)
  full_mae = mean_absolute_error(Y_true,Y_pred)

  dates = list(get_date()[0])

  dates.pop(0)

  data = {'date': pd.to_datetime(dates, format='%Y-%m-%d'), 'Y_true': Y_true, 'Y_pred': Y_pred}
  df = pd.DataFrame(data)
  interval = 30 if df.shape[0]>100 else 3
  percentile_x = np.percentile(df['date'], args_train_len*100)

  # Plot the data
  plt.figure(figsize=(10, 6))
  plt.plot(df['date'], df['Y_true'], label='Y_true', marker='o')
  plt.plot(df['date'], df['Y_pred'], label='Y_pred', marker='x')
  plt.axvline(percentile_x, color='red', linestyle='--', label=f'Vertical Line at {int(args_train_len*100)}%')
  plt.xlabel('Date')
  plt.ylabel('Values')
  plt.title(f'Y_true vs Y_pred for {args_data} {scalar_type_name}')
  plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
  plt.legend()
  plt.grid()
  plt.plot()
  plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=interval))  #change it to 30 if data size is more and 3 if data size is less
  plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

  # Specify the folder path and filename for saving the plot
  folder_path = f'Graphs/{train_test_size}/{scalar_type_name}/'
  file_name = f'{row+1}-{args_data}.png'

  create_folder(folder_path)

  # Combine folder path and file name
  file_path = folder_path + file_name

  # Save the plot to the specified file path
  plt.savefig(file_path)

In [ ]:
def create_results():

  run_train_pred()
  run_val_pred()
  run_test_pred()
  run_full_pred()

  # save result to csv
  data = {
      'data': [args_data],
      'model': [args_model],
      'seq_len': [args_seq_len],
      'pred_len': [args_pred_len],
      'lr': [args_learning_rate],
      'mse': [test_mse],
      'rmse': [test_rmse],
      'median': [test_median],
      'mae': [test_mae],
      'val_mse': [val_mse],
      'val_rmse': [val_rmse],
      'val_median': [val_median],
      'val_mae': [val_mae],
      'train_mse': [train_mse],
      'train_rmse': [train_rmse],
      'train_median': [train_median],
      'train_mae': [train_mae],
      'training_time': [elasped_training_time],
      'norm_type': [args_norm_type],
      'train_test_size': train_test_size,
      'scalar': scalar_type_name,
      'activation': args_activation,
      'n_block': args_n_block,
      'dropout': args_dropout,
      'batch_size':args_batch_size,
      'ff_dim': args_ff_dim
  }

  df = pd.DataFrame(data)
  if os.path.exists(args_result_path):
    df.to_csv(args_result_path, mode='a', index=False, header=False)
  else:
    df.to_csv(args_result_path, mode='w', index=False, header=True)

In [ ]:
def train_tsmixer():
  tf.random.set_seed(args_seed)
  np.random.seed(args_seed)
  random.seed(args_seed)
  tf.keras.utils.set_random_seed(args_seed)

  global train_data, val_data, test_data, full_data, train_median, val_median, test_median, full_median, elasped_training_time, n_feature, target_slice

  # load datasets
  data_loader = TSFDataLoader(
      args_data,
      args_batch_size,
      args_seq_len,
      args_pred_len,
      args_feature_type,
      args_train_len,
      args_scaler_type,
      args_target,
  )
  n_feature = data_loader.n_feature
  target_slice = data_loader.target_slice

  train_data = data_loader.get_train(True)
  val_data = data_loader.get_val()
  test_data = data_loader.get_test()
  full_data = data_loader.get_full()

  train_median, val_median, test_median, full_median = data_loader.get_median()

  # train model

  build_model = tsmixer.build_model
  model = build_model(
      input_shape = (args_seq_len, n_feature),
      pred_len = args_pred_len,
      norm_type = args_norm_type,
      activation = args_activation,
      dropout = args_dropout,
      n_block = args_n_block,
      ff_dim = args_ff_dim,
      target_slice = target_slice,
  )


  def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

  optimizer = tf.keras.optimizers.Adam(learning_rate=args_learning_rate)
  model.compile(optimizer=optimizer, loss='mse', metrics=[rmse,'mae','msle'])

  checkpoint_path = os.path.join(args_checkpoint_dir, f'{exp_id}_best')

  checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_path,
      verbose=1,
      save_best_only=True,
      save_weights_only=True,
  )

  early_stop_callback = tf.keras.callbacks.EarlyStopping(
      monitor='val_loss', patience=args_patience
  )

  start_training_time = time.time()

  history = model.fit(
      train_data,
      epochs=args_train_epochs,
      validation_data=val_data,
      callbacks=[checkpoint_callback, early_stop_callback],
  )

  end_training_time = time.time()
  elasped_training_time = end_training_time - start_training_time
  print(f'Training finished in {elasped_training_time} seconds')

  # evaluate best model
  best_epoch = np.argmin(history.history['val_loss'])
  model.load_weights(checkpoint_path)
  test_result = model.evaluate(test_data)
  if args_delete_checkpoint:
    for f in glob.glob(checkpoint_path + '*'):
      os.remove(f)


In [ ]:
config_df = pd.read_csv("config_values.csv")
for row in range(config_df.shape[0]):
  args_data = config_df.loc[row,'args_data']
  args_model = config_df.loc[row,'args_model']
  args_train_epochs = config_df.loc[row,'args_train_epochs']
  args_batch_size = int(config_df.loc[row,'args_batch_size'])  # Reduce batch size to prevent overfitting with a small dataset
  args_seq_len = int(config_df.loc[row,'args_seq_len'])
  args_pred_len = int(config_df.loc[row,'args_pred_len'])
  args_feature_type = config_df.loc[row,'args_feature_type']
  args_target = config_df.loc[row,'args_target']
  args_norm_type = config_df.loc[row,'args_norm_type']
  args_activation = config_df.loc[row,'args_activation']
  args_train_len = config_df.loc[row,'args_train_len']
  args_scaler_type = config_df.loc[row,'args_scaler_type'] # Use 'U' for un normalize 'S' for StandardScaler and 'M' for MinMaxScaler
  args_dropout = config_df.loc[row,'args_dropout']  # Increase dropout for more regularization
  args_n_block = int(config_df.loc[row,'args_n_block'])
  args_ff_dim = int(config_df.loc[row,'args_ff_dim'])  # Reduce feedforward dimension for simplicity
  args_kernel_size = int(config_df.loc[row,'args_kernel_size'])
  args_learning_rate = config_df.loc[row,'args_learning_rate']   # Experiment with a slightly higher learning rate
  args_patience = int(config_df.loc[row,'args_patience'])  # Increase patience for early stopping
  args_seed = int(config_df.loc[row,'args_seed'])
  scalar_type_name = config_df.loc[row,'scalar_type_name']
  train_test_size = f"{int((args_train_len)*100)}_{int(round(1-args_train_len,2)*100)}"

  substrings_to_check = ['g_cases_worldwide_2yr']

  if(any(substring in args_data for substring in substrings_to_check)):
    df = pd.read_csv(f"/content/dataset/{args_data}.csv")
    df["g_values"] = df["g_values"].fillna(0)
    df = df.loc[:,['date','g_values']]
    df.to_csv(f"/content/dataset/{args_data}.csv")


  exp_id = f'{args_data}_{args_feature_type}_{args_model}_sl{args_seq_len}_pl{args_pred_len}_lr{args_learning_rate}_nt{args_norm_type}_{args_activation}_nb{args_n_block}_dp{args_dropout}_fd{args_ff_dim}_tl{args_train_len}_st{args_scaler_type}'
  print(exp_id)
  train_tsmixer()
  create_results()


In [ ]:
!zip -r Graphs.zip Graphs

  adding: Graphs/ (stored 0%)
  adding: Graphs/70_30/ (stored 0%)
  adding: Graphs/70_30/Unnormalized/ (stored 0%)
  adding: Graphs/70_30/Unnormalized/40-KBest_agg_min_test.png (deflated 6%)
  adding: Graphs/70_30/Unnormalized/64-RFE_agg_avg.png (deflated 5%)
  adding: Graphs/70_30/Unnormalized/28-KBest_agg_max_test.png (deflated 6%)
  adding: Graphs/70_30/Unnormalized/16-KBest_agg_avg.png (deflated 5%)
  adding: Graphs/70_30/Unnormalized/40-KBest_agg_min.png (deflated 5%)
  adding: Graphs/70_30/Unnormalized/64-RFE_agg_avg_test.png (deflated 6%)
  adding: Graphs/70_30/Unnormalized/28-KBest_agg_max.png (deflated 5%)
  adding: Graphs/70_30/Unnormalized/100-RFE_agg_sum.png (deflated 4%)
  adding: Graphs/70_30/Unnormalized/76-RFE_agg_max.png (deflated 5%)
  adding: Graphs/70_30/Unnormalized/4-g_cases_worldwide_2yr_test.png (deflated 7%)
  adding: Graphs/70_30/Unnormalized/100-RFE_agg_sum_test.png (deflated 7%)
  adding: Graphs/70_30/Unnormalized/88-RFE_agg_min.png (deflated 5%)
  adding: G

In [ ]:
!cp Graphs.zip "/content/drive/MyDrive/Colab Notebooks/ML Projects/Jeevitha/TSMixer/Ts_Mixer_Final/6-28 Jan 2024/"

In [ ]:
!cp result.csv "/content/drive/MyDrive/Colab Notebooks/ML Projects/Jeevitha/TSMixer/Ts_Mixer_Final/6-28 Jan 2024/"

In [ ]:
result = pd.read_csv('result.csv')

In [ ]:
grouped_data = result.groupby('data')

# Find the row with the minimum 'mse' value within each group
min_mse_rows = grouped_data.apply(lambda x: x[x['mse'] == x['mse'].min()])

# Reset the index of the resulting DataFrame
min_mse_df = min_mse_rows.reset_index(drop=True)

# Print the resulting DataFrame
min_mse_df

,data,model,seq_len,pred_len,lr,mse,rmse,median,mae,val_mse,...,train_mae,training_time,norm_type,train_test_size,scalar,activation,n_block,dropout,batch_size,ff_dim
0,KBest_agg_avg,tsmixer,1,1,0.0001,0.006461,0.080380,0.087373,0.045442,0.004508,...,0.175104,6.353856,B,75_25,MinMaxScaler,relu,2,0.1,32,128
1,KBest_agg_max,tsmixer,1,1,0.0001,0.005392,0.073433,0.074937,0.041154,0.004469,...,0.228014,7.080881,B,60_40,MinMaxScaler,relu,2,0.1,32,128
2,KBest_agg_min,tsmixer,1,1,0.0001,0.006419,0.080119,0.074937,0.052242,0.006412,...,0.215821,6.402406,B,60_40,MinMaxScaler,relu,2,0.5,90,128
3,KBest_agg_sum,tsmixer,1,1,0.0001,0.034797,0.186540,0.368477,0.155488,0.188913,...,0.600340,7.939994,B,80_20,Unnormalized,relu,2,0.5,90,128
4,RFE_agg_avg,tsmixer,1,1,0.0001,0.006304,0.079401,0.087373,0.068442,0.003952,...,0.252935,7.719372,B,75_25,MinMaxScaler,relu,2,0.1,32,128
5,RFE_agg_max,tsmixer,1,1,0.0001,0.005343,0.073095,0.074937,0.052060,0.004461,...,0.205913,8.008025,B,60_40,MinMaxScaler,relu,2,0.1,32,128
6,RFE_agg_min,tsmixer,1,1,0.0001,0.006304,0.079401,0.087373,0.068442,0.003952,...,0.252935,8.215232,B,75_25,MinMaxScaler,relu,2,0.1,32,128
7,RFE_agg_sum,tsmixer,1,1,0.0001,0.007774,0.088171,0.106392,0.071314,0.000264,...,0.245915,10.522468,B,80_20,MinMaxScaler,relu,2,0.1,32,128
8,g_cases_worldwide_2yr,tsmixer,1,1,0.0100,0.004520,0.067233,-0.002983,0.034561,0.001209,...,0.101549,7.881076,B,70_30,MinMaxScaler,relu,2,0.1,32,128
9,g_cases_worldwide_3mth,tsmixer,1,1,0.0001,0.005615,0.074935,0.074937,0.041682,0.004100,...,0.164300,34.032626,L,60_40,MinMaxScaler,relu,2,0.1,32,128


In [ ]:
substrings_to_check = ['g_cases_worldwide_2yr', 'g_cases_worldwide_3mth']

# Check if 'data' column contains any substring from the list
mask = min_mse_df['data'].str.contains('|'.join(substrings_to_check))

# Invert the mask using the ~ operator to keep only rows where 'data' does not contain any of the specified substrings
result_filtered = min_mse_df[~mask]

# Print the resulting DataFrame
result_filtered

,data,model,seq_len,pred_len,lr,mse,rmse,median,mae,val_mse,...,train_mae,training_time,norm_type,train_test_size,scalar,activation,n_block,dropout,batch_size,ff_dim
0,KBest_agg_avg,tsmixer,1,1,0.0001,0.006461,0.080380,0.087373,0.045442,0.004508,...,0.175104,6.353856,B,75_25,MinMaxScaler,relu,2,0.1,32,128
1,KBest_agg_max,tsmixer,1,1,0.0001,0.005392,0.073433,0.074937,0.041154,0.004469,...,0.228014,7.080881,B,60_40,MinMaxScaler,relu,2,0.1,32,128
2,KBest_agg_min,tsmixer,1,1,0.0001,0.006419,0.080119,0.074937,0.052242,0.006412,...,0.215821,6.402406,B,60_40,MinMaxScaler,relu,2,0.5,90,128
3,KBest_agg_sum,tsmixer,1,1,0.0001,0.034797,0.186540,0.368477,0.155488,0.188913,...,0.600340,7.939994,B,80_20,Unnormalized,relu,2,0.5,90,128
4,RFE_agg_avg,tsmixer,1,1,0.0001,0.006304,0.079401,0.087373,0.068442,0.003952,...,0.252935,7.719372,B,75_25,MinMaxScaler,relu,2,0.1,32,128
5,RFE_agg_max,tsmixer,1,1,0.0001,0.005343,0.073095,0.074937,0.052060,0.004461,...,0.205913,8.008025,B,60_40,MinMaxScaler,relu,2,0.1,32,128
6,RFE_agg_min,tsmixer,1,1,0.0001,0.006304,0.079401,0.087373,0.068442,0.003952,...,0.252935,8.215232,B,75_25,MinMaxScaler,relu,2,0.1,32,128
7,RFE_agg_sum,tsmixer,1,1,0.0001,0.007774,0.088171,0.106392,0.071314,0.000264,...,0.245915,10.522468,B,80_20,MinMaxScaler,relu,2,0.1,32,128


In [ ]:
result_filtered[result_filtered['mse']==result_filtered['mse'].min()]

,data,model,seq_len,pred_len,lr,mse,rmse,median,mae,val_mse,...,train_mae,training_time,norm_type,train_test_size,scalar,activation,n_block,dropout,batch_size,ff_dim
5,RFE_agg_max,tsmixer,1,1,0.0001,0.005343,0.073095,0.074937,0.05206,0.004461,...,0.205913,8.008025,B,60_40,MinMaxScaler,relu,2,0.1,32,128


In [ ]:
print(f'rmse is {round(0.00737*100/0.7343,4)} % of median')

rmse is 1.0037 % of median


In [ ]:
substrings_to_check = ['g_cases_worldwide_3mth']

# Check if 'data' column contains any substring from the list
mask = result['data'].str.contains('|'.join(substrings_to_check))

# Invert the mask using the ~ operator to keep only rows where 'data' does not contain any of the specified substrings
gdata_3month_filtered = result[mask]

# Print the resulting DataFrame
gdata_3month_filtered

,data,model,seq_len,pred_len,lr,mse,rmse,median,mae,val_mse,...,train_mae,training_time,norm_type,train_test_size,scalar,activation,n_block,dropout,batch_size,ff_dim
108,g_cases_worldwide_3mth,tsmixer,1,1,0.0001,0.038209,0.195471,0.329667,0.176344,0.033920,...,0.297844,33.481401,L,60_40,Unnormalized,relu,2,0.1,32,128
109,g_cases_worldwide_3mth,tsmixer,1,1,0.0001,0.273021,0.522514,-0.905879,0.501216,0.270913,...,0.756649,14.045068,L,60_40,StandardScaler,relu,2,0.1,32,128
110,g_cases_worldwide_3mth,tsmixer,1,1,0.0001,0.005615,0.074935,0.074937,0.041682,0.004100,...,0.164300,34.032626,L,60_40,MinMaxScaler,relu,2,0.1,32,128
111,g_cases_worldwide_3mth,tsmixer,1,1,0.0001,0.042697,0.206633,0.339350,0.186332,0.033056,...,0.313507,43.721331,L,70_30,Unnormalized,relu,2,0.1,32,128
112,g_cases_worldwide_3mth,tsmixer,1,1,0.0001,0.204045,0.451713,-0.740732,0.435168,0.266134,...,0.850077,34.001646,L,70_30,StandardScaler,relu,2,0.1,32,128
113,g_cases_worldwide_3mth,tsmixer,1,1,0.0001,0.006761,0.082227,0.082785,0.046731,0.005217,...,0.193251,33.969682,L,70_30,MinMaxScaler,relu,2,0.1,32,128
114,g_cases_worldwide_3mth,tsmixer,1,1,0.0001,0.044673,0.211359,0.345011,0.190319,0.035450,...,0.288173,36.845525,L,75_25,Unnormalized,relu,2,0.1,32,128
115,g_cases_worldwide_3mth,tsmixer,1,1,0.0001,0.189926,0.435806,-0.680224,0.417296,0.181302,...,0.813144,33.216921,L,75_25,StandardScaler,relu,2,0.1,32,128
116,g_cases_worldwide_3mth,tsmixer,1,1,0.0001,0.007346,0.085712,0.087373,0.048767,0.004306,...,0.180408,33.515820,L,75_25,MinMaxScaler,relu,2,0.1,32,128
117,g_cases_worldwide_3mth,tsmixer,1,1,0.0001,0.042988,0.207335,0.368477,0.183106,0.013945,...,0.283472,34.681727,L,80_20,Unnormalized,relu,2,0.1,32,128


In [ ]:
gdata_3month_filtered[gdata_3month_filtered['mse']==gdata_3month_filtered['mse'].min()]

,data,model,seq_len,pred_len,lr,mse,rmse,median,mae,val_mse,...,train_mae,training_time,norm_type,train_test_size,scalar,activation,n_block,dropout,batch_size,ff_dim
110,g_cases_worldwide_3mth,tsmixer,1,1,0.0001,0.005615,0.074935,0.074937,0.041682,0.0041,...,0.1643,34.032626,L,60_40,MinMaxScaler,relu,2,0.1,32,128


In [ ]:
print(f'rmse is {round(0.339398*100/0.7343,4)} % of median')

rmse is 46.2206 % of median


In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 2830.556549549103 seconds ---


In [ ]:
2830.556549549103/60

47.17594249248505